In [1]:
import gymnasium as gym
import random
import numpy as np
import math

In [2]:
env = gym.make("CliffWalking-v0",render_mode="human")

In [3]:
print(env.observation_space)
print(env.action_space)
action_space = [0,1,2,3]

Discrete(48)
Discrete(4)


In [4]:
# #Random testing
# episodes = 1
# for episode in range(0,episodes):
#     state = np.array(env.reset()[0])
#     done = False
#     score = 0
#     steps = 0
#     while not done and steps < 100:
#         action = random.choice(action_space)
#         observation, reward, done,_,_  = env.step(action)
#         steps+=1
#         score+=reward
#         env.render()
    
#     print(f"Episode {episode+1} : score = {score}")
# env.close()

In [5]:
StateGrid = [[i*12 + j for j in range(12)]for i in range(4)]
print(StateGrid)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23], [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]]


In [6]:
discount = 0.7

In [7]:
def ingrid(i,j):
    if i < 0 or j < 0 or j >= 12 or i >= 4:
        return False
    return True

def next_states(state):
    i = state//12
    j = state%12
    new_state = []
    #up
    if ingrid(i-1,j): new_state.append(StateGrid[i-1][j]) 
    #down
    if ingrid(i+1,j): new_state.append(StateGrid[i+1][j])
    #left
    if ingrid(i,j-1): new_state.append(StateGrid[i][j-1])
    #right
    if ingrid(i,j+1): new_state.append(StateGrid[i][j+1])
    
    
    return new_state

In [8]:
def reward_model(state):
    if(state == 47):
        return 1.0
    elif state <= 36:
        return -1.0
    elif 37 <= state <= 46:
        return -100.0
    else: 
        return -1.0
    


def iteration(values):

    def future_reward(nextStates, newState):
        val = float(values[newState])
        return val

    convergence = 0
    new_values = [-1*math.inf]*48
    for row in StateGrid:
        for state in row:
            # #print(state)
            # if 37 <= state <= 46:
            #     new_values[state] = -100.0
            # elif state == 47:
            #     new_values[state] = values[state] + reward_model(state)
            #else:
            nextStates = next_states(state)
            for newState in nextStates:
                new_values[state] = max(new_values[state], reward_model(state) + discount*(future_reward(nextStates,newState)))
            convergence = max(convergence,abs(new_values[state]-values[state]))

    
    return (new_values , convergence)

In [9]:
state_values = [float(0)]*48
delta = 1
it = 0
while delta > 0.01 and 10000 > it:
    state_values , delta = iteration(state_values)
    it += 1

print(it)
for i in range(4):
    for j in range(12):
        print(state_values[StateGrid[i][j]] , end=" ")
    print()

14
-3.3107258975716998 -3.2913480954902994 -3.2830433231696996 -3.2518015606302995 -3.2265482733696995 -3.171094346630299 -3.1112522533697 -3.0063857466302997 -2.8759542533696996 -2.6702457466303002 -2.3957542533696996 -1.9842457466302998 
-3.2913480954902994 -3.2830433231696996 -3.2518015606302995 -3.2265482733696995 -3.171094346630299 -3.1112522533697 -3.0063857466302997 -2.8759542533696996 -2.6702457466303002 -2.3957542533696996 -1.9842457466302998 -1.4157542533697 
-3.2830433231696996 -3.2518015606302995 -3.2265482733696995 -3.171094346630299 -3.1112522533697 -3.0063857466302997 -2.8759542533696996 -2.6702457466303002 -2.3957542533696996 -1.9842457466302998 -1.4157542533697 -0.5842457466302999 
-3.2913480954902994 -102.2830433231697 -102.2518015606303 -102.2265482733697 -102.1710943466303 -102.1112522533697 -102.0063857466303 -101.8759542533697 -101.6702457466303 -101.3957542533697 -99.5842457466303 0.5842457466302999 


In [ ]:
def predict_action(state):
    val = float(-1*math.inf)
    action_state = -1
    for states in next_states(state):
        if(state == -1): continue
        if val < state_values[states]:
            val = state_values[states]
            action_state = states 

    #left
    if action_state + 1 == state:
        return 3
    #down
    elif action_state - 12 == state:
        return 2
    #right
    elif action_state - 1 == state:
        return 1
    
    return 0

#Random testing
episodes = 1
for episode in range(0,episodes):
    state = np.array(env.reset()[0])
    done = False
    score = 0
    steps = 0
    while not done and steps < 100:
        action = predict_action(state)
        state, reward, done,_,_  = env.step(action)
        steps+=1
        score+=reward
        env.render()
    
    print(f"Episode {episode+1} : score = {score}")
env.close()